In [2]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, merge, Input
from keras.layers import LSTM, Dropout, GRU
from keras.callbacks import ModelCheckpoint
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import re
import string
import codecs

## Friends dataset

In [2]:
path = filename = "transcripts/friends_transcript.txt"
text_left = open(path).read().lower()
text_left = ''.join(filter(string.printable.__contains__,text_left))
text_left = re.sub(r'[\r\n{}#"_`]',' ',text_left.strip())
text_left = text_left.strip()
chars_left = sorted(list(set(text_left)))

In [3]:
len(chars_left)

54

## TBBT dataset

In [23]:
path = filename = "transcripts/tbbt_transcript.txt"
text_right = open(path).read().lower()
text_right = ''.join(filter(string.printable.__contains__,text_right))
text_right = re.sub(r'[\n\xa0]',' ',text_right.strip())
chars_right = sorted(list(set(text_right)))

In [24]:
len(chars_right)

54

In [3]:
def vectorization(text,chars):
    print('total chars:', len(chars))
    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))

    # cut the text in semi-redundant sequences of maxlen characters
    maxlen = 40
    step = 3
    sentences = []
    next_chars = []
    for i in range(0, len(text) - maxlen, step):
        sentences.append(text[i: i + maxlen])
        next_chars.append(text[i + maxlen])
    print('nb sequences:', len(sentences))
    return maxlen, char_indices, indices_char

In [28]:
maxlen, char_indices_left, indices_char_left = vectorization(text_left, chars_left)
maxlen, char_indices_right, indices_char_right = vectorization(text_right, chars_right)

total chars: 54
nb sequences: 1261555
total chars: 54
nb sequences: 1133085


In [29]:
left = Sequential()
left.add(LSTM(256, input_shape=(maxlen, len(chars_left)),return_sequences=True))
left.add(LSTM(128, input_shape=(maxlen, len(chars_left))))
left.add(Dense(len(chars_left)))
left.add(Activation('softmax'))
filename = "friends_weights/weights-improvement-00-1.0237-full_data.hdf5"
left.load_weights(filename)
optimizer = RMSprop(lr=0.0001)
left.compile(loss='categorical_crossentropy', optimizer=optimizer)

right = Sequential()
right.add(LSTM(256, input_shape=(maxlen, len(chars_right)),return_sequences=True))
right.add(LSTM(128, input_shape=(maxlen, len(chars_right))))
right.add(Dense(len(chars_right)))
right.add(Activation('softmax'))
filename = "tbbt_weights/weights-improvement-00-0.9772-full_data_700.hdf5"
right.load_weights(filename)
optimizer = RMSprop(lr=0.001)
right.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [4]:
def sample(preds, temperature=1.0):
    '''helper function to sample an index from a probability array'''
    
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [26]:
def text_gen(sentence, model, chars, char_indices, indices_char, maxlen=40):
    '''Function prints dialogues, predicted one character (letter) at a time'''
    
    generated = ''
    generated += sentence
    for i in range(100):
        x = np.zeros((1, maxlen, len(chars)))

        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.


        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, 0.6)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()
    return sentence

# Script mashup

In [33]:
try:
    generated = ''
    start_index = random.randint(0, len(text_left) - maxlen - 1)
    sentence = text_left[start_index: start_index + maxlen]
    sys.stdout.write(generated)
    while True:
        sentence_left = text_gen(sentence, left, chars_left, char_indices_left, indices_char_left)
        sentence_right = text_gen(sentence_left, right, chars_right, char_indices_right, indices_char_right)
        sentence = sentence_right
except KeyboardInterrupt:
    pass

t, and 

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


i was just a couple of cleans in the matter with me? rachel:  so you were there and get the casting here for a good time you were right.    joey: alright monica, i was thinking about you? monica:  hey! joey:  yeah! ross:  okay, i got you to wear, they have to be the work on a fine, i like her and be in england  and then i can be a girls doesn't seem some kind of best friends all the time is
 my way to see how to do the season and the end of the train of lithers and player and do it and also reading a bad jerk for the store astronaut. leonard: i dont know what to get to be a bad guiders. we can be able to have a solution of his signation of the warm discovery like a little connection. leonard: thats the strange equive company. amy: i think were not an actual time. i was sure youre try
ing to let you see the coffee. ross:  oh i thought i love you out of my life.    rachel: i know. i mean, i want to get me for a little buttomera.    rachel: you look good, just be pregnant and they just go

# Talking to Joey

In [6]:
path = "friends_characters/joey_dialogues.txt"
with codecs.open(path, encoding='utf-8') as f:
    text = f.read().lower()
    text = ''.join(filter(string.printable.__contains__,text))
#     text = re.sub(r'\n',' ',text.strip())
    text = text.strip()

In [7]:
chars = sorted(list(set(text)))

In [8]:
maxlen, char_indices, indices_char = vectorization(text, chars)

total chars: 52
nb sequences: 137149


In [10]:
left = Sequential()
left.add(LSTM(256, input_shape=(maxlen, len(chars)),return_sequences=True))
left.add(LSTM(128, input_shape=(maxlen, len(chars))))
left.add(Dense(len(chars)))
left.add(Activation('softmax'))
filename = 'friends_weights/weights-improvement-00-0.3446-joey.hdf5'
left.load_weights(filename)
optimizer = RMSprop(lr=0.001)
left.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [1]:
sentence = 'hey joey how are you doing?'

In [2]:
len(sentence)

27

In [3]:
if len(sentence) < 40:
    sentence += (40-len(sentence))*' '

In [4]:
sentence

'hey joey how are you doing?             '

In [ ]:
text_gen(sentence, left, chars, char_indices, indices_char)

ight ev

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


er gonna get to meet the thing. im after thing out of the cares?
oh well you got it.  we c